In [20]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from textblob import TextBlob as tb
from datetime import datetime
import numpy as np

In [14]:
def readxls(filename):
    xl = pd.ExcelFile(filename)
    tb = xl.parse()
    var = ['Title','PublicationDate','DocumentURL']
    data = tb[var]
    return (data)

In [15]:
def readtxt(filename):
    with open(filename, 'r', encoding = 'utf-8') as file:
        text = file.read()
    post = text.split("____________________________________________________________")
    post = post[1:]
    post = post[:-1]
    texts = []
    for i in range (len(post)):
        num_text = post[i].find("Full text:")
        num_sub = post[i].find("Subject:")
        text = post[i][num_text+len("Full text: "):num_sub]
        texts.append(text)
    textdf = pd.DataFrame(texts)
    textdf.columns = ["Full Text"]
    return(textdf)

In [16]:
def getsenti(mylist):
    senti = []
    for i in range(len(mylist)):
        blob = tb(mylist[i])
        value = blob.sentiment.polarity
        senti.append(value)
    return(senti)

In [17]:
def timesenti(df):
    textlist = df["Full Text"].tolist()
    sentiment = getsenti(textlist)
    sentidata = pd.concat([df["PublicationDate"], pd.DataFrame(sentiment)], axis=1)
    timesenti = sentidata.groupby("PublicationDate", axis=0).mean()
    timesenti.index = pd.to_datetime(timesenti.index)
    timesenti["date"]=timesenti.index
    my_list = list(timesenti.index)
    my_list.sort()
    df = pd.DataFrame(np.array(my_list), columns = ["date"])
    s = pd.merge(df,timesenti, how='left', on=['date'])
    return(s)

In [18]:
Google2016Janxl = readxls("Google_2016Jan.xls")
Google2016Jantxt = readtxt("Google_2016Jan.txt")
Google2016Jan = pd.concat([Google2016Janxl,Google2016Jantxt], axis=1)

In [21]:
timesenti2016Jan = timesenti(Google2016Jan)

In [24]:
xlsfiles = ["0.xls","1.xls","2.xls","3.xls","4.xls","5.xls","6.xls","7.xls","8.xls","9.xls","10.xls","11.xls","12.xls"]
txtfiles = ["0.txt","1.txt","2.txt","3.txt","4.txt","5.txt","6.txt","7.txt","8.txt","9.txt","10.txt","11.txt","12.txt"]

In [25]:
xls = {file: readxls(file) for file in xlsfiles}

In [32]:
txt = {file: readtxt(file) for file in txtfiles}

In [47]:
data = {i : pd.concat([xls[xlsfiles[i]],txt[txtfiles[i]]], axis=1) for i in range(13)}

In [52]:
senti = {i: timesenti(data[i]) for i in range(13)}

In [57]:
sentiseries = senti[0]
sentiseries = sentiseries.append(senti[1], ignore_index = True)

In [65]:
sentiseries = senti[0]
for i in range(1,13):
    sentiseries = sentiseries.append(senti[i], ignore_index = True)

In [66]:
sentiseries

,date,0
0,2016-01-03,0.116100
1,2016-01-04,0.097372
2,2016-01-05,0.119200
3,2016-01-06,0.100297
4,2016-01-07,0.062459
5,2016-01-08,0.153506
6,2016-01-09,0.107977
7,2016-01-10,0.098012
8,2016-01-11,0.082656
9,2016-01-12,0.125590


In [68]:
sentiseries.set_index('date')

,0
date,
2016-01-03,0.116100
2016-01-04,0.097372
2016-01-05,0.119200
2016-01-06,0.100297
2016-01-07,0.062459
2016-01-08,0.153506
2016-01-09,0.107977
2016-01-10,0.098012
2016-01-11,0.082656
